In [ ]:
# ============================================================================
# DEEP FAKE VIDEO DETECTION - ANN THESIS PROJECT
# CRISP-DM Methodology Implementation
# Dataset: Deep Fake Detection (DFD) Dataset from HuggingFace
# ============================================================================

"""
PROJECT STRUCTURE:
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling (Multiple Approaches)
5. Evaluation
6. Deployment/Results

Author: Thesis Student
Date: 2025
"""

# ============================================================================
# STEP 0: ENVIRONMENT SETUP
# ============================================================================

# Install required packages
!pip install -q datasets huggingface_hub
!pip install -q opencv-python-headless
!pip install -q scikit-learn
!pip install -q matplotlib seaborn
!pip install -q pillow
!pip install -q torch torchvision torchaudio
!pip install -q timm  # PyTorch Image Models
!pip install -q grad-cam  # For explainability

# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms, models
import timm

# Computer Vision
import cv2
from PIL import Image

# ML & Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report,
                             roc_curve, auc, roc_auc_score)

# HuggingFace
from datasets import load_dataset

# Utilities
from tqdm.auto import tqdm
import time
from datetime import datetime

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ============================================================================
# STEP 1: BUSINESS UNDERSTANDING
# ============================================================================

"""
PROJECT OBJECTIVE:
Develop an Artificial Neural Network model to detect deepfake videos with:
- High accuracy (>90%)
- Good generalization
- Interpretability/Explainability
- Comparison of multiple architectures

BUSINESS VALUE:
- Combat misinformation
- Protect digital identity
- Ensure media authenticity

SUCCESS CRITERIA:
- Accuracy > 90%
- F1-Score > 0.88
- Low false negative rate (don't miss deepfakes)
- Model explainability through visualization
"""

print("="*80)
print("DEEP FAKE DETECTION PROJECT - BUSINESS UNDERSTANDING")
print("="*80)
print("\nObjective: Detect deepfake videos using deep learning")
print("Target Metric: Accuracy > 90%, F1-Score > 0.88")
print("Approach: Compare multiple CNN architectures + Transfer Learning")
print("="*80)

# ============================================================================
# STEP 2: DATA UNDERSTANDING
# ============================================================================

print("\n" + "="*80)
print("STEP 2: DATA UNDERSTANDING - LOADING DATASET")
print("="*80)

# Load dataset from HuggingFace
print("\n[INFO] Loading dataset from HuggingFace...")
print("This may take several minutes depending on dataset size...")

try:
    # Load the dataset
    dataset = load_dataset("Hemgg/deep-fake-detection-dfd-entire-original-dataset")
    
    print("\n✓ Dataset loaded successfully!")
    print(f"\nDataset structure: {dataset}")
    
    # Explore the dataset
    print("\n" + "-"*80)
    print("DATASET EXPLORATION")
    print("-"*80)
    
    # Check available splits
    print(f"\nAvailable splits: {list(dataset.keys())}")
    
    # Get the training data
    train_data = dataset['train']
    
    print(f"\nNumber of samples in train split: {len(train_data)}")
    print(f"\nColumn names: {train_data.column_names}")
    print(f"\nFeatures: {train_data.features}")
    
    # Show first sample
    print("\n" + "-"*80)
    print("SAMPLE DATA INSPECTION")
    print("-"*80)
    sample = train_data[0]
    print("\nFirst sample keys:", sample.keys())
    for key, value in sample.items():
        if key == 'video':
            print(f"\n{key}: <Video data - shape/type: {type(value)}>")
        else:
            print(f"{key}: {value}")
    
except Exception as e:
    print(f"\n✗ Error loading dataset: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check internet connection")
    print("2. Verify HuggingFace dataset URL")
    print("3. Try: !huggingface-cli login")

# ============================================================================
# EXPLORATORY DATA ANALYSIS (EDA)
# ============================================================================

print("\n" + "="*80)
print("EXPLORATORY DATA ANALYSIS (EDA)")
print("="*80)

# Function to analyze dataset distribution
def analyze_dataset(dataset, split_name='train'):
    """Perform comprehensive EDA on the dataset"""
    
    print(f"\n[INFO] Analyzing {split_name} split...")
    
    # Convert to pandas for easier analysis
    df = pd.DataFrame(dataset)
    
    print("\n1. BASIC STATISTICS")
    print("-"*80)
    print(f"Total samples: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nData types:\n{df.dtypes}")
    
    # Check for label column
    label_columns = [col for col in df.columns if 'label' in col.lower()]
    
    if label_columns:
        label_col = label_columns[0]
        print(f"\n2. LABEL DISTRIBUTION (Column: {label_col})")
        print("-"*80)
        
        label_counts = df[label_col].value_counts()
        print(f"\n{label_counts}")
        
        # Calculate percentages
        label_percentages = df[label_col].value_counts(normalize=True) * 100
        print(f"\nPercentages:\n{label_percentages}")
        
        # Visualize distribution
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Bar plot
        label_counts.plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
        axes[0].set_title('Label Distribution (Count)', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Label', fontsize=12)
        axes[0].set_ylabel('Count', fontsize=12)
        axes[0].tick_params(axis='x', rotation=0)
        
        # Pie chart
        colors = ['#2ecc71', '#e74c3c']
        axes[1].pie(label_counts, labels=label_counts.index, autopct='%1.1f%%',
                   colors=colors, startangle=90)
        axes[1].set_title('Label Distribution (Percentage)', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('label_distribution.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Check for class imbalance
        imbalance_ratio = label_counts.max() / label_counts.min()
        print(f"\n3. CLASS IMBALANCE ANALYSIS")
        print("-"*80)
        print(f"Imbalance Ratio: {imbalance_ratio:.2f}")
        if imbalance_ratio > 1.5:
            print("⚠ Warning: Significant class imbalance detected!")
            print("  → Consider using weighted loss or oversampling")
        else:
            print("✓ Classes are relatively balanced")
    
    # Check for missing values
    print(f"\n4. MISSING VALUES")
    print("-"*80)
    missing = df.isnull().sum()
    if missing.sum() > 0:
        print(missing[missing > 0])
    else:
        print("✓ No missing values found")
    
    return df

# Run EDA
try:
    df_analysis = analyze_dataset(train_data, 'train')
except Exception as e:
    print(f"Error during EDA: {e}")

# ============================================================================
# STEP 3: DATA PREPARATION
# ============================================================================

print("\n" + "="*80)
print("STEP 3: DATA PREPARATION")
print("="*80)

# Custom Dataset Class for Video/Image Data
class DeepfakeDataset(Dataset):
    """Custom Dataset for Deepfake Detection"""
    
    def __init__(self, hf_dataset, transform=None, max_samples=None):
        """
        Args:
            hf_dataset: HuggingFace dataset
            transform: torchvision transforms
            max_samples: Limit number of samples (for testing)
        """
        self.dataset = hf_dataset
        self.transform = transform
        
        if max_samples:
            self.dataset = self.dataset.select(range(min(max_samples, len(self.dataset))))
        
        # Identify label column
        self.label_col = None
        for col in self.dataset.column_names:
            if 'label' in col.lower():
                self.label_col = col
                break
        
        print(f"[INFO] Dataset initialized with {len(self.dataset)} samples")
        if self.label_col:
            print(f"[INFO] Using '{self.label_col}' as label column")
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        
        # Extract image/video frame
        # This depends on your dataset structure - adapt as needed
        if 'video' in sample:
            # For video data, extract first frame
            video = sample['video']
            if isinstance(video, dict) and 'path' in video:
                image = self._load_video_frame(video['path'])
            else:
                # Handle other video formats
                image = self._extract_frame(video)
        elif 'image' in sample:
            image = sample['image']
            if not isinstance(image, Image.Image):
                image = Image.fromarray(image)
        else:
            # Fallback: try to find any image-like data
            for key, value in sample.items():
                if isinstance(value, (Image.Image, np.ndarray)):
                    image = value if isinstance(value, Image.Image) else Image.fromarray(value)
                    break
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        # Get label
        if self.label_col:
            label = sample[self.label_col]
            # Convert string labels to integers if needed
            if isinstance(label, str):
                label = 1 if label.lower() in ['fake', 'deepfake', '1'] else 0
        else:
            label = 0  # Default if no label found
        
        return image, label
    
    def _load_video_frame(self, video_path):
        """Load first frame from video file"""
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        cap.release()
        
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            return Image.fromarray(frame)
        else:
            # Return black image if failed
            return Image.new('RGB', (224, 224))
    
    def _extract_frame(self, video_data):
        """Extract frame from video data"""
        # Implement based on your video data format
        # This is a placeholder
        return Image.new('RGB', (224, 224))

# Define data transforms
print("\n[INFO] Defining data transformations...")

# Training transforms (with augmentation)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

# Validation/Test transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225])
])

print("✓ Transforms defined")
print("\nTraining augmentations:")
print("  - Resize to 224x224")
print("  - Random horizontal flip")
print("  - Random rotation (±10°)")
print("  - Color jitter")
print("  - Normalization (ImageNet stats)")

# Create datasets
print("\n[INFO] Creating datasets...")

# Start with smaller sample for testing
MAX_SAMPLES_TEST = 100  # Set to None for full dataset

try:
    # Create full dataset
    full_dataset = DeepfakeDataset(
        train_data, 
        transform=train_transform,
        max_samples=MAX_SAMPLES_TEST
    )
    
    # Split into train/val/test (70/15/15)
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(SEED)
    )
    
    # Apply validation transform to val and test sets
    val_dataset.dataset.transform = val_transform
    test_dataset.dataset.transform = val_transform
    
    print(f"\n✓ Datasets created successfully!")
    print(f"  - Training samples: {len(train_dataset)}")
    print(f"  - Validation samples: {len(val_dataset)}")
    print(f"  - Test samples: {len(test_dataset)}")
    
except Exception as e:
    print(f"\n✗ Error creating datasets: {e}")
    print("\nNote: Adjust the dataset loading based on actual data structure")

# Create data loaders
print("\n[INFO] Creating data loaders...")

BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=2,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"✓ Data loaders created")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Training batches: {len(train_loader)}")
print(f"  - Validation batches: {len(val_loader)}")
print(f"  - Test batches: {len(test_loader)}")

# Visualize sample batch
print("\n[INFO] Visualizing sample batch...")

def show_batch(dataloader, n_images=8):
    """Display a batch of images with labels"""
    batch = next(iter(dataloader))
    images, labels = batch
    
    # Denormalize images
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    images = images * std + mean
    images = torch.clamp(images, 0, 1)
    
    # Plot
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for i in range(min(n_images, len(images))):
        img = images[i].permute(1, 2, 0).cpu().numpy()
        axes[i].imshow(img)
        label_text = "FAKE" if labels[i].item() == 1 else "REAL"
        color = 'red' if labels[i].item() == 1 else 'green'
        axes[i].set_title(f'Label: {label_text}', fontsize=12, 
                         fontweight='bold', color=color)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.savefig('sample_batch.png', dpi=300, bbox_inches='tight')
    plt.show()

try:
    show_batch(train_loader)
    print("✓ Sample batch visualization saved")
except Exception as e:
    print(f"Could not visualize batch: {e}")

# ============================================================================
# STEP 4: MODELING
# ============================================================================

print("\n" + "="*80)
print("STEP 4: MODELING - BUILDING NEURAL NETWORKS")
print("="*80)

# Model 1: Simple CNN (Baseline)
class SimpleCNN(nn.Module):
    """Simple CNN baseline model"""
    
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        
        self.features = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Model 2: ResNet18 (Transfer Learning)
def create_resnet18(num_classes=2, pretrained=True):
    """Create ResNet18 model with custom classifier"""
    model = models.resnet18(pretrained=pretrained)
    
    # Freeze early layers (optional)
    # for param in model.parameters():
    #     param.requires_grad = False
    
    # Replace final layer
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    
    return model

# Model 3: EfficientNet-B0 (State-of-the-art)
def create_efficientnet(num_classes=2, pretrained=True):
    """Create EfficientNet-B0 model"""
    model = timm.create_model('efficientnet_b0', pretrained=pretrained)
    
    # Replace classifier
    num_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    
    return model

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs=10, device='cuda', model_name='model'):
    """
    Train a model and track metrics
    
    Returns:
        model: Trained model
        history: Training history dictionary
    """
    
    print(f"\n[INFO] Training {model_name}...")
    print(f"Device: {device}")
    print(f"Epochs: {num_epochs}")
    print(f"Optimizer: {optimizer.__class__.__name__}")
    print(f"Learning Rate: {optimizer.param_groups[0]['lr']}")
    print("-"*80)
    
    model = model.to(device)
    
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [],
        'epoch_times': []
    }
    
    best_val_acc = 0.0
    best_model_wts = model.state_dict()
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 40)
        
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc='Training')
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == labels).sum().item()
            train_total += labels.size(0)
            
            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        epoch_train_loss = train_loss / train_total
        epoch_train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc='Validation')
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                
                val_pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100 * val_correct / val_total:.2f}%'
                })
        
        epoch_val_loss = val_loss / val_total
        epoch_val_acc = val_correct / val_total
        
        epoch_time = time.time() - epoch_start
        
        # Save history
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        history['epoch_times'].append(epoch_time)
        
        # Print epoch summary
        print(f'\nEpoch Summary:')
        print(f'  Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc*100:.2f}%')
        print(f'  Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc*100:.2f}%')
        print(f'  Time: {epoch_time:.2f}s')
        
        # Save best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_model_wts = model.state_dict()
            print(f'  ✓ New best model! (Val Acc: {best_val_acc*100:.2f}%)')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    
    print(f'\n{"="*80}')
    print(f'Training completed!')
    print(f'Best validation accuracy: {best_val_acc*100:.2f}%')
    print(f'{"="*80}\n')
    
    return model, history

# Evaluation function
def evaluate_model(model, test_loader, device='cuda', model_name='model'):
    """
    Evaluate model on test set and return detailed metrics
    """
    
    print(f"\n[INFO] Evaluating {model_name} on test set...")
    
    model.eval()
    model = model.to(device)
    
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # ROC AUC
    try:
        roc_auc = roc_auc_score(all_labels, all_probs[:, 1])
    except:
        roc_auc = 0.0
    
    # Print results
    print(f"\n{'='*80}")
    print(f"TEST RESULTS - {model_name}")
    print(f"{'='*80}")
    print(f"\nOverall Metrics:")
    print(f"  Accuracy:  {accuracy*100:.2f}%")
    print(f"  Precision: {precision*100:.2f}%")
    print(f"  Recall:    {recall*100:.2f}%")
    print(f"  F1-Score:  {f1*100:.2f}%")
    print(f"  ROC-AUC:   {roc_auc:.4f}")
    
    print(f"\nDetailed Classification Report:")
    print(classification_report(all_labels, all_preds, 
                                target_names=['REAL', 'FAKE']))
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['REAL', 'FAKE'],
                yticklabels=['REAL', 'FAKE'])
    plt.title(f'Confusion Matrix - {model_name}', fontsize=14, fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{model_name}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Plot ROC curve
    if roc_auc > 0:
        fpr, tpr, _ = roc_curve(all_labels, all_probs[:, 1])
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, linewidth=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title(f'ROC Curve - {model_name}', fontsize=14, fontweight='bold')
        plt.legend(fontsize=10)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'roc_curve_{model_name}.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm
    }
    
    return metrics

# Visualization function for training history
def plot_training_history(history, model_name='model'):
    """Plot training and validation metrics"""
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss plot
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Training Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[0].set_title(f'Training and Validation Loss - {model_name}', 
                     fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].legend(fontsize=10)
    axes[0].grid(alpha=0.3)
    
    # Accuracy plot
    axes[1].plot(epochs, [acc*100 for acc in history['train_acc']], 
                'b-', label='Training Accuracy', linewidth=2)
    axes[1].plot(epochs, [acc*100 for acc in history['val_acc']], 
                'r-', label='Validation Accuracy', linewidth=2)
    axes[1].set_title(f'Training and Validation Accuracy - {model_name}', 
                     fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy (%)', fontsize=12)
    axes[1].legend(fontsize=10)
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'training_history_{model_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

# ============================================================================
# TRAINING MODELS
# ============================================================================

print("\n" + "="*80)
print("TRAINING MULTIPLE MODELS")
print("="*80)

# Dictionary to store all results
all_results = {}

# Hyperparameters
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

# Model 1: Simple CNN
print("\n" + "="*80)
print("MODEL 1: SIMPLE CNN (BASELINE)")
print("="*80)

try:
    simple_cnn = SimpleCNN(num_classes=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(simple_cnn.parameters(), lr=LEARNING_RATE)
    
    simple_cnn, history_simple = train_model(
        simple_cnn, train_loader, val_loader, 
        criterion, optimizer, 
        num_epochs=NUM_EPOCHS, 
        device=device,
        model_name='SimpleCNN'
    )
    
    plot_training_history(history_simple, 'SimpleCNN')
    metrics_simple = evaluate_model(simple_cnn, test_loader, device, 'SimpleCNN')
    all_results['SimpleCNN'] = metrics_simple
    
except Exception as e:
    print(f"Error training SimpleCNN: {e}")

# Model 2: ResNet18
print("\n" + "="*80)
print("MODEL 2: RESNET18 (TRANSFER LEARNING)")
print("="*80)

try:
    resnet18 = create_resnet18(num_classes=2, pretrained=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)
    
    resnet18, history_resnet = train_model(
        resnet18, train_loader, val_loader,
        criterion, optimizer,
        num_epochs=NUM_EPOCHS,
        device=device,
        model_name='ResNet18'
    )
    
    plot_training_history(history_resnet, 'ResNet18')
    metrics_resnet = evaluate_model(resnet18, test_loader, device, 'ResNet18')
    all_results['ResNet18'] = metrics_resnet
    
except Exception as e:
    print(f"Error training ResNet18: {e}")

# Model 3: EfficientNet-B0
print("\n" + "="*80)
print("MODEL 3: EFFICIENTNET-B0 (STATE-OF-THE-ART)")
print("="*80)

try:
    efficientnet = create_efficientnet(num_classes=2, pretrained=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(efficientnet.parameters(), lr=LEARNING_RATE)
    
    efficientnet, history_eff = train_model(
        efficientnet, train_loader, val_loader,
        criterion, optimizer,
        num_epochs=NUM_EPOCHS,
        device=device,
        model_name='EfficientNet-B0'
    )
    
    plot_training_history(history_eff, 'EfficientNet-B0')
    metrics_eff = evaluate_model(efficientnet, test_loader, device, 'EfficientNet-B0')
    all_results['EfficientNet-B0'] = metrics_eff
    
except Exception as e:
    print(f"Error training EfficientNet: {e}")

# ============================================================================
# STEP 5: MODEL COMPARISON & SELECTION
# ============================================================================

print("\n" + "="*80)
print("STEP 5: MODEL COMPARISON & SELECTION")
print("="*80)

# Create comparison DataFrame
comparison_df = pd.DataFrame({
    model_name: {
        'Accuracy (%)': metrics['accuracy'] * 100,
        'Precision (%)': metrics['precision'] * 100,
        'Recall (%)': metrics['recall'] * 100,
        'F1-Score (%)': metrics['f1_score'] * 100,
        'ROC-AUC': metrics['roc_auc']
    }
    for model_name, metrics in all_results.items()
}).T

print("\n" + "-"*80)
print("MODEL COMPARISON TABLE")
print("-"*80)
print(comparison_df.to_string())

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_to_plot = ['Accuracy (%)', 'Precision (%)', 'Recall (%)', 'F1-Score (%)']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    comparison_df[metric].plot(kind='bar', ax=ax, color=colors)
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('Model', fontsize=12)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', padding=3)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Select best model
best_model_name = comparison_df['F1-Score (%)'].idxmax()
best_f1_score = comparison_df.loc[best_model_name, 'F1-Score (%)']

print(f"\n{'='*80}")
print(f"BEST MODEL SELECTED: {best_model_name}")
print(f"F1-Score: {best_f1_score:.2f}%")
print(f"{'='*80}")

# ============================================================================
# STEP 6: HYPERPARAMETER TUNING (Optional - on best model)
# ============================================================================

print("\n" + "="*80)
print("STEP 6: HYPERPARAMETER TUNING")
print("="*80)
print("\n[INFO] Hyperparameter tuning can be performed on the best model")
print("Consider tuning:")
print("  - Learning rate")
print("  - Batch size")
print("  - Optimizer (Adam, SGD, AdamW)")
print("  - Number of epochs")
print("  - Dropout rates")
print("  - Data augmentation parameters")
print("\nThis step is implemented in a separate notebook for grid/random search")

# ============================================================================
# STEP 7: FINAL REPORT & DOCUMENTATION
# ============================================================================

print("\n" + "="*80)
print("PROJECT SUMMARY REPORT")
print("="*80)

print(f"\n{'='*80}")
print("DEEPFAKE DETECTION PROJECT - FINAL SUMMARY")
print(f"{'='*80}")
print(f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Dataset: Deep Fake Detection (DFD) from HuggingFace")
print(f"Total Samples: {len(train_data)}")
print(f"Train/Val/Test Split: 70/15/15")
print(f"\nModels Trained: {len(all_results)}")
print(f"Best Model: {best_model_name}")
print(f"Best F1-Score: {best_f1_score:.2f}%")

print(f"\n{'='*80}")
print("KEY FINDINGS")
print(f"{'='*80}")
print("\n1. Data Quality:")
print("   - Dataset loaded successfully from HuggingFace")
print("   - Class distribution analyzed")
print("   - Data augmentation applied")

print("\n2. Model Performance:")
print(comparison_df.to_string())

print("\n3. Recommendations:")
if best_f1_score >= 90:
    print("   ✓ Model meets target performance (F1 > 90%)")
else:
    print("   → Consider additional training epochs")
    print("   → Try ensemble methods")
    print("   → Collect more training data")

print("\n4. Next Steps:")
print("   - Deploy best model")
print("   - Implement real-time video detection")
print("   - Add explainability (Grad-CAM)")
print("   - Write IEEE conference paper")

print(f"\n{'='*80}")
print("PROJECT COMPLETED SUCCESSFULLY!")
print(f"{'='*80}\n")

# Save results
results_file = 'model_results.txt'
with open(results_file, 'w') as f:
    f.write("="*80 + "\n")
    f.write("DEEPFAKE DETECTION - MODEL RESULTS\n")
    f.write("="*80 + "\n\n")
    f.write(comparison_df.to_string())
    f.write(f"\n\nBest Model: {best_model_name}\n")
    f.write(f"Best F1-Score: {best_f1_score:.2f}%\n")

print(f"\n[INFO] Results saved to '{results_file}'")

# ============================================================================
# END OF SCRIPT
# ============================================================================

print("\n" + "="*80)
print("ALL TASKS COMPLETED!")
print("="*80)
print("\nGenerated Files:")
print("  - label_distribution.png")
print("  - sample_batch.png")
print("  - training_history_*.png")
print("  - confusion_matrix_*.png")
print("  - roc_curve_*.png")
print("  - model_comparison.png")
print("  - model_results.txt")
print("\nNext: Review results and proceed with IEEE paper writing")
print("="*80)